In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint, uniform
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
import joblib


In [6]:
data = pd.read_csv('data/features/features.csv')

In [7]:
data = data[['similaridade_cv_competencias','similaridade_cv_atividades','avaliador_idioma_ingles','avaliador_idioma_espanhol','situacao_candidado_avaliador']]

In [8]:
data['similaridade'] = ''
data['similaridade'].loc[data['similaridade_cv_atividades']>=data['similaridade_cv_atividades']] = data['similaridade_cv_atividades'].loc[data['similaridade_cv_atividades']>=data['similaridade_cv_atividades']]
data['similaridade'].loc[data['similaridade_cv_atividades']<data['similaridade_cv_atividades']] = data['similaridade_cv_atividades'].loc[data['similaridade_cv_atividades']<data['similaridade_cv_atividades']]
data['situacao_candidado_avaliador'] = pd.qcut(data['situacao_candidado_avaliador'], q=3, labels=['baixo', 'medio', 'alto'])
balance = pd.concat([data.loc[data['situacao_candidado_avaliador'] == 'alto'].sample(n=1000, random_state=42), data.loc[data['situacao_candidado_avaliador'] == 'medio'].sample(n=1000, random_state=42), data.loc[data['situacao_candidado_avaliador'] == 'baixo'].sample(n=1000, random_state=42)])

/tmp/ipykernel_673942/1987047533.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['similaridade'].loc[data['similaridade_cv_atividades']>=data['similaridade_cv_atividades']] = data['similaridade_cv_atividades'].loc[data['similaridade

In [9]:
X = balance[['similaridade','avaliador_idioma_ingles','avaliador_idioma_espanhol']]
# y = pd.qcut(balance['situacao_candidado_avaliador'], q=3, labels=['baixo', 'medio', 'alto'])  
y = balance['situacao_candidado_avaliador']
le = LabelEncoder()
y_encoded = le.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
# baixo = 1, medio = 2, alto = 0

In [10]:
Counter(y)

Counter({'alto': 1000, 'medio': 1000, 'baixo': 1000})

In [217]:
# # Pipeline com XGBoost
# pipeline_xgb = Pipeline([
#     ('scaler', StandardScaler()),
#     ('clf', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
# ])

# # Treinar
# pipeline_xgb.fit(X_train, y_train)

# # Avaliar
# print("XGBoost Results:")
# y_pred_xgb = pipeline_xgb.predict(X_test)
# print(classification_report(y_test, y_pred_xgb))

In [218]:
# # Pipeline com LightGBM
# pipeline_lgb = Pipeline([
#     ('smote', SMOTE(random_state=42)),
#     ('scaler', StandardScaler()),
#     ('clf', LGBMClassifier(class_weight='balanced', random_state=42))
# ])
# param_grid = {
#     'clf__n_estimators': [400],
#     'clf__learning_rate': [0.1],
#     'clf__max_depth': [15],
# }
# grid_search = GridSearchCV(
#     estimator=pipeline_lgb,
#     param_grid=param_grid,
#     scoring='f1_macro',  
#     cv=3,
#     n_jobs=-1,
#     verbose=2
# )

# # Treinar
# grid_search.fit(X_train, y_train)

# # Resultados
# print("✅ Melhor combinação de hiperparâmetros:")
# print(grid_search.best_params_)

# print("\n📊 Avaliação no conjunto de teste:")

# y_pred = grid_search.predict(X_test)
# print(classification_report(le.inverse_transform(y_test), le.inverse_transform(y_pred)))

In [11]:
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', RandomForestClassifier(class_weight='balanced', random_state=42))
])
pipeline_rf.fit(X_train, y_train)
print(classification_report(y_test, pipeline_rf.predict(X_test)))


              precision    recall  f1-score   support

           0       0.42      0.37      0.39       217
           1       0.36      0.39      0.38       186
           2       0.38      0.39      0.39       197

    accuracy                           0.39       600
   macro avg       0.39      0.39      0.38       600
weighted avg       0.39      0.39      0.39       600



In [12]:
Counter(y_train)

Counter({np.int64(1): 814, np.int64(2): 803, np.int64(0): 783})

In [229]:
from sklearn.model_selection import StratifiedKFold

# Pipeline
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),  # Remove se só tiver variáveis categóricas
    ('clf', RandomForestClassifier(class_weight='balanced', random_state=42))
])

# Parâmetros para GridSearch
param_grid_rf = {
    'clf__n_estimators': [250],
    'clf__max_depth': [20],
    'clf__min_samples_split': [2],
    'clf__min_samples_leaf': [1],
}

# Validação cruzada estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Grid Search
grid_search_rf = GridSearchCV(
    pipeline_rf,
    param_grid_rf,
    cv=cv,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=2
)

# Treinar
grid_search_rf.fit(X_train, y_train)

# Melhor modelo
best_rf = grid_search_rf.best_estimator_
print("Melhores parâmetros:", grid_search_rf.best_params_)

# Avaliar
y_pred = best_rf.predict(X_test)
print("\nRelatório de classificação no teste:")
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END clf__max_depth=20, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=250; total time=   0.6s
[CV] END clf__max_depth=20, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=250; total time=   0.6s
[CV] END clf__max_depth=20, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=250; total time=   0.6s
[CV] END clf__max_depth=20, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=250; total time=   0.7s
[CV] END clf__max_depth=20, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=250; total time=   0.7s
Melhores parâmetros: {'clf__max_depth': 20, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 250}

Relatório de classificação no teste:
              precision    recall  f1-score   support

           0       0.43      0.39      0.41       217
           1       0.37      0.39      0.38       186
        

In [223]:
# Modelos base
estimators = [
    ('rf', RandomForestClassifier(class_weight='balanced', random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ('lgbm', LGBMClassifier(random_state=42))
]

# Meta-modelo
meta_model = LogisticRegression(max_iter=1000, random_state=42)

# Stacking Classifier
stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1,
    passthrough=False  # ou True se quiser incluir as features originais no meta-modelo
)

# Pipeline completo
pipeline_stacking = Pipeline([
    ('scaler', StandardScaler()),
    ('stacking', stacking_clf)
])

# Parâmetros para GridSearchCV (focando no meta-modelo)
param_grid = {
    'stacking__final_estimator__C': [0.1, 1.0, 10],
    'stacking__final_estimator__penalty': ['l2'],
    'stacking__final_estimator__solver': ['lbfgs', 'saga']
}

# GridSearchCV
grid = GridSearchCV(
    pipeline_stacking,
    param_grid=param_grid,
    cv=3,
    scoring='f1_weighted',
    n_jobs=-1,
    verbose=2
)

# Treinar
grid.fit(X_train, y_train)

# Avaliar
y_pred = grid.predict(X_test)
print("\n📊 Relatório de Classificação - StackingClassifier:")
print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.262234 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 268
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.309515 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 267
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.311645 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 268
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.269342 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 268
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.335005 seconds.
You can set `force_col_wise=true` to remove the overhead

[23:04:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[23:04:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[23:04:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[23:04:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[23:04:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.272148 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 268
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 3
[LightGBM] [Info] Start training from score -1.120091
[LightGBM] [Info] Start training from score -1.080650
[LightGBM] [Info] Start training from score -1.095492


[23:04:51] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.323066 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 268
[LightGBM] [Info] Number of data points in the train set: 1600, number of used features: 3
[LightGBM] [Info] Start training from score -1.120091
[LightGBM] [Info] Start training from score -1.082493
[LightGBM] [Info] Start training from score -1.093625


...

/home/thomaz/APP_5/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/thomaz/APP_5/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/thomaz/APP_5/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[CV] END stacking__final_estimator__C=1.0, stacking__final_estimator__penalty=l2, stacking__final_estimator__solver=saga; total time=531.3min
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000334 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 268
[LightGBM] [Info] Number of data points in the train set: 2400, number of used features: 3
[LightGBM] [Info] Start training from score -1.120091
[LightGBM] [Info] Start training from score -1.081264
[LightGBM] [Info] Start training from score -1.094869


/home/thomaz/APP_5/venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [07:56:16] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[07:56:17] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[07:56:17] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[07:56:17] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[07:56:17] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[07:56:17] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001276 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 268
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 3
[LightGBM] [Info] Start training from score -1.120730
[LightGBM] [Info] Start training from score -1.081571
[LightGBM] [Info] Start training from score -1.093936
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001587 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 268
[LightGBM] [Info] Number of data points in the train set: 1920, number of used features: 3
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001043 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 267
[LightGBM] [Info] Number of data points in the train set: 1920, number 

/home/thomaz/APP_5/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/thomaz/APP_5/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/thomaz/APP_5/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/thomaz/APP_5/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/thomaz/APP_5/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  

In [230]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.43      0.39      0.41       217
           1       0.37      0.39      0.38       186
           2       0.38      0.39      0.39       197

    accuracy                           0.39       600
   macro avg       0.39      0.39      0.39       600
weighted avg       0.39      0.39      0.39       600



In [234]:
grid_search_rf
joblib.dump(grid_search_rf, 'data/modelos/modelo_stacking.pkl')

['data/modelos/modelo_stacking.pkl']

In [14]:
data.loc[data['situacao_candidado_avaliador'] == 'alto']

,similaridade_cv_competencias,similaridade_cv_atividades,avaliador_idioma_ingles,avaliador_idioma_espanhol,situacao_candidado_avaliador,similaridade
33,0.069556,0.037299,0.50000,1.0,alto,0.037299
55,0.060914,0.060914,0.25000,1.0,alto,0.060914
59,0.078220,0.078220,0.12500,1.0,alto,0.07822
64,0.083174,0.083174,0.03125,1.0,alto,0.083174
67,0.106975,0.106975,0.12500,1.0,alto,0.106975
...,...,...,...,...,...,...
56462,0.024075,-0.005898,0.50000,0.5,alto,-0.005898
56466,0.024075,-0.005898,0.50000,0.5,alto,-0.005898
56490,0.082033,0.082033,0.06250,1.0,alto,0.082033
56499,0.051174,-0.015143,0.12500,0.5,alto,-0.015143


In [15]:
X_train

,similaridade,avaliador_idioma_ingles,avaliador_idioma_espanhol
45577,0.091137,0.500,0.5
30514,0.015254,1.000,1.0
23584,0.07959,1.000,1.0
47566,0.09122,1.000,1.0
45483,0.036322,1.000,1.0
...,...,...,...
16923,-0.024306,0.125,0.5
5936,-0.0372,0.500,0.5
1927,-0.007223,0.500,1.0
48877,0.090499,0.125,1.0
